used dataset: https://www.kaggle.com/datasets/ishantjuyal/emotions-in-text

In [1]:
!pip install bigartm10

     |████████████████████████████████| 2.0 MB 14.1 MB/s 


In [2]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 8.2 MB 32.8 MB/s 


In [3]:
import artm
artm.ARTM(num_topics=1)
print(artm.version())

0.10.1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_path = '/content/drive/MyDrive/artm_emotion_detection/'

In [6]:
import nltk
import pymorphy2

nltk.download('stopwords')

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [41]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, accuracy_score

In [8]:
emotion_df = pd.read_csv(data_path+'Emotion_final.csv')
emotion_df.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


# Preprocessing

In [9]:
import string

def filter_and_tokenize(text):
    text = text.lower().replace('\n', ' ')

    text = ' '.join([t for t in text.split(' ')
                        if not t.startswith('@') and not t.startswith('http') and not t.startswith('www')])

    for p in string.punctuation:
        text = text.replace(p, ' ')

    return [t for t in text.split(' ') if len(t) > 1]

In [10]:
emotion_df['Text'] = emotion_df['Text'].apply(filter_and_tokenize)
emotion_df.head()

,Text,Emotion
0,"[didnt, feel, humiliated]",sadness
1,"[can, go, from, feeling, so, hopeless, to, so,...",sadness
2,"[im, grabbing, minute, to, post, feel, greedy,...",anger
3,"[am, ever, feeling, nostalgic, about, the, fir...",love
4,"[am, feeling, grouchy]",anger


In [11]:
emotion_df.shape

(21459, 2)

In [12]:
from time import time
# lemmatization
lemmatizer = pymorphy2.MorphAnalyzer()

def lemmatize(tokens):
    return [lemmatizer.parse(t)[0].normal_form for t in tokens]

t = time()
emotion_df['Text'] = emotion_df['Text'].apply(lemmatize)
dt = time() - t
print(dt)
emotion_df.head()

/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)


19.43777298927307


,Text,Emotion
0,"[didnt, feel, humiliated]",sadness
1,"[can, go, from, feeling, so, hopeless, to, so,...",sadness
2,"[im, grabbing, minute, to, post, feel, greedy,...",anger
3,"[am, ever, feeling, nostalgic, about, the, fir...",love
4,"[am, feeling, grouchy]",anger


In [13]:
stopwords = set(nltk.corpus.stopwords.words('english'))

def remove_stopwords(tokens):
    return [t for t in tokens if t not in stopwords]

emotion_df['Text'] = emotion_df['Text'].apply(remove_stopwords)

In [14]:
from collections import Counter

In [15]:
token_to_counter = Counter()
for row in emotion_df['Text']:
    token_to_counter.update(row)
token_to_counter.most_common(30)

[('feel', 13973),
 ('feeling', 6461),
 ('like', 3661),
 ('im', 3055),
 ('really', 1201),
 ('know', 1091),
 ('time', 1032),
 ('little', 960),
 ('get', 947),
 ('people', 865),
 ('would', 847),
 ('want', 811),
 ('one', 802),
 ('still', 759),
 ('even', 746),
 ('think', 744),
 ('ive', 723),
 ('life', 694),
 ('make', 656),
 ('bit', 650),
 ('something', 646),
 ('much', 643),
 ('could', 627),
 ('love', 620),
 ('going', 616),
 ('things', 615),
 ('dont', 596),
 ('way', 593),
 ('day', 558),
 ('back', 540)]

the words: 'feel', 'feeling', looks like fillers

In [16]:
new_stopwords = set(['feel', 'feeling', 'im'])
def remove_new_stopwords(tokens):
    return [t for t in tokens if t not in new_stopwords]
emotion_df['Text'] = emotion_df['Text'].apply(remove_new_stopwords)
emotion_df.head()

,Text,Emotion
0,"[didnt, humiliated]",sadness
1,"[go, hopeless, damned, hopeful, around, someon...",sadness
2,"[grabbing, minute, post, greedy, wrong]",anger
3,"[ever, nostalgic, fireplace, know, still, prop...",love
4,[grouchy],anger


In [17]:
emotion_df.shape

(21459, 2)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train_emotion_df, test_emotion_df = train_test_split(emotion_df, test_size=0.2, stratify=emotion_df['Emotion'], random_state=1)

In [20]:
vw_filename_train = 'train_texts.vw.txt'
counter = 0
with open(data_path + vw_filename_train, 'w') as fout:
    for row in train_emotion_df.iterrows():
      tokens = row[1][0]
      label = row[1][1]
      fout.write('doc_{} {} |@label class_{}\n'.format(counter, " ".join(tokens), label))
      counter += 1

# ARTM try

In [21]:
bv_train = artm.BatchVectorizer(data_path=data_path + vw_filename_train, 
                          data_format='vowpal_wabbit', 
                          batch_size=2000, 
                          target_folder='batches')

In [24]:
def get_score_artm(n_topics, decor_tau, sparse_tau, passes):
  model = artm.ARTM(num_topics=n_topics, num_document_passes=3, dictionary=bv_train.dictionary, class_ids={'@default_class': 1.0})
  # scores
  model.scores.add(artm.PerplexityScore(name='perplexity', dictionary=bv_train.dictionary))
  model.scores.add(artm.TopTokensScore(name='top-tokens', num_tokens=20))
  model.scores.add(artm.SparsityPhiScore(name='sparsity'))
  # regs
  model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator', tau=decor_tau))
  # fit artm
  model.fit_offline(bv_train, num_collection_passes=(passes + 1)//2)
  model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='smooth', tau=sparse_tau,
                                                          dictionary=bv_train.dictionary,
                                                          topic_names=model.topic_names))
  model.fit_offline(bv_train, num_collection_passes=(passes + 1)//2)
  phi = model.get_phi()
  phi.index = phi.index.map(lambda it: it[1])
  # get train dataset for rf
  train_y = train_emotion_df['Emotion']
  train_df = pd.DataFrame(index=train_emotion_df.index, columns=phi.columns)
  
  for i in train_df.index:
    tmp = (phi.loc[train_emotion_df.loc[i]['Text']]*n_topics).apply(np.log1p).sum(axis=0).apply(np.exp)
    if (tmp.sum()):
      tmp /= tmp.sum()
    train_df.loc[i] = tmp
  # fit rf
  rf = RandomForestClassifier(random_state=1, n_estimators=n_topics*2, max_depth=5)
  rf.fit(train_df, train_y)
  pred = rf.predict(train_df)
  # test model
  test_y = test_emotion_df['Emotion']
  test_df = pd.DataFrame(index=test_emotion_df.index, columns=phi.columns)
  for i in test_df.index:
    idx = test_emotion_df.loc[i]['Text']
    idx = [j for j in idx if j in phi.index]
    tmp = (phi.loc[idx]*n_topics).apply(np.log1p).sum(axis=0).apply(np.exp)
    if (tmp.sum()):
      tmp /= tmp.sum()
    test_df.loc[i] = tmp
  test_pred = rf.predict(test_df)
  return {'micro f1': f1_score(test_y, test_pred, average='micro'), 
          'macro f1': f1_score(test_y, test_pred, average='macro'), 
          'accuracy': rf.score(test_df, test_y)}

In [25]:
# WARNIG: very long
for n_topics in [100, 200, 400]:
  for decor_tau in [0, 5e+3, 5e+4, 5e+5]:
    for sparse_tau in [0, 50, 100, 200]:
      t = time()
      print(n_topics, decor_tau, sparse_tau, get_score_artm(n_topics, decor_tau, sparse_tau, 40))
      print(f'^--- time: {time()-t}')

100 0 0 {'micro f1': 0.35834109972041006, 'macro f1': 0.13329784598064773, 'accuracy': 0.35834109972041006}
^--- time: 127.5278947353363
100 0 50 {'micro f1': 0.3616029822926375, 'macro f1': 0.1350412690172315, 'accuracy': 0.3616029822926375}
^--- time: 129.42705154418945
100 0 100 {'micro f1': 0.36020503261882575, 'macro f1': 0.1341476448852971, 'accuracy': 0.36020503261882575}
^--- time: 129.90519428253174
100 0 200 {'micro f1': 0.36206896551724144, 'macro f1': 0.1360577152340682, 'accuracy': 0.3620689655172414}
^--- time: 128.57873797416687
100 5000.0 0 {'micro f1': 0.3818732525629078, 'macro f1': 0.15628795234367898, 'accuracy': 0.3818732525629077}
^--- time: 126.17937207221985
100 5000.0 50 {'micro f1': 0.3702236719478098, 'macro f1': 0.13960026256297883, 'accuracy': 0.37022367194780986}
^--- time: 129.62586307525635
100 5000.0 100 {'micro f1': 0.3706896551724138, 'macro f1': 0.14053376368556422, 'accuracy': 0.3706896551724138}
^--- time: 129.61424231529236
100 5000.0 200 {'micro 

# Tensorflow lstm try

In [26]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
max_dict_size = 200000
text_tok = Tokenizer(num_words=max_dict_size, oov_token="<OOV>")

text_tok.fit_on_texts(emotion_df['Text'])

In [28]:
seq = pad_sequences(text_tok.texts_to_sequences(emotion_df['Text']), padding='post')
train_seq = seq[train_emotion_df.index]
test_seq = seq[test_emotion_df.index]

In [29]:
test_seq.shape

(4292, 34)

In [30]:
y_train = train_emotion_df['Emotion']
encoder = dict(zip(y_train.unique(), range(6)))
y_train = y_train.replace(encoder)
y_test = test_emotion_df['Emotion'].replace(encoder)

In [31]:
y_train_np = np.zeros((y_train.shape[0], 6))
for i in range(y_train_np.shape[0]):
  y_train_np[i, y_train.iloc[i]] = 1
y_test_np = np.zeros((y_test.shape[0], 6))
for i in range(y_test_np.shape[0]):
  y_test_np[i, y_test.iloc[i]] = 1

In [38]:
model = tf.keras.Sequential([
  layers.Embedding(max_dict_size, 32),
  tf.keras.layers.LSTM(32),
  layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=['accuracy'])

In [39]:
es = tf.keras.callbacks.EarlyStopping(
    patience=40,
    restore_best_weights=True
)

In [40]:
model.fit(train_seq, y_train_np, 
          epochs=100, batch_size=1024, 
          validation_data=(test_seq, y_test_np),
          callbacks=[es]
          )

Epoch 1/100
17/17 [==============================] - 7s 259ms/step - loss: 1.7316 - accuracy: 0.3275 - val_loss: 1.6444 - val_accuracy: 0.3276
Epoch 2/100
17/17 [==============================] - 4s 217ms/step - loss: 1.6174 - accuracy: 0.3275 - val_loss: 1.5956 - val_accuracy: 0.3276
Epoch 3/100
17/17 [==============================] - 4s 218ms/step - loss: 1.5891 - accuracy: 0.3275 - val_loss: 1.5850 - val_accuracy: 0.3276
Epoch 4/100
17/17 [==============================] - 4s 222ms/step - loss: 1.5838 - accuracy: 0.3275 - val_loss: 1.5832 - val_accuracy: 0.3276
Epoch 5/100
17/17 [==============================] - 4s 219ms/step - loss: 1.5807 - accuracy: 0.3275 - val_loss: 1.5790 - val_accuracy: 0.3276
Epoch 6/100
17/17 [==============================] - 4s 219ms/step - loss: 1.5599 - accuracy: 0.3275 - val_loss: 1.5136 - val_accuracy: 0.3276
Epoch 7/100
17/17 [==============================] - 4s 221ms/step - loss: 1.3105 - accuracy: 0.3513 - val_loss: 1.2852 - val_accuracy: 0.4091

In [42]:
pred = model.predict(test_seq)

In [43]:
pred_compressed = np.argmax(pred, axis=1)

In [45]:
print('micro f1', f1_score(y_test, pred_compressed, average='micro'))
print('macro f1', f1_score(y_test, pred_compressed, average='macro'))
print('macro f1', accuracy_score(y_test, pred_compressed))

micro f1 0.7593196644920783
macro f1 0.6754285476516406
macro f1 0.7593196644920783
